# Spark SQL Demo

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('My First Spark application') \
    .getOrCreate() 

sc = spark.sparkContext

Read in some data:

In [2]:
business = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_business.json.gz')

In [3]:
checkin = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_checkin.json.gz')
review = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_review.json.gz')
tip = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_tip.json.gz')
user = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_user.json.gz')

Look at a schema:

In [4]:
tip.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- compliment_count: long (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)



and some data:

In [5]:
tips = tip.first().asDict()

In [6]:
for k,v in tips.items():
    print(f'{k}:\t{v}')

business_id:	VaKXUpmWTTWDKbpJ3aQdMw
compliment_count:	0
date:	2014-03-27 03:51:24
text:	Great for watching games, ufc, and whatever else tickles yer fancy
user_id:	UPw5DWs_b-e2JRBS-t37Ag


Create a temporary view and issue a SQL query:

In [7]:
tip.createOrReplaceTempView("tip")
result = spark.sql("SELECT COUNT(*) FROM tip")

Show the results:

In [8]:
result.show()

+--------+
|count(1)|
+--------+
| 1223094|
+--------+



Does it match what we would expect?

In [9]:
tip.count()

1223094

Go ahead and create temp views for all our DataFrames:

In [10]:
business.createOrReplaceTempView("business")
checkin.createOrReplaceTempView("checkin")
tip.createOrReplaceTempView("tip")
review.createOrReplaceTempView("review")
user.createOrReplaceTempView("user")

## Simple SQL query

In [11]:
result = spark.sql("SELECT * FROM business WHERE state='QC'")
result.show()

+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|               city|               hours|is_open|     latitude|     longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|8261 Saint-Lauren...|[,, u'full_bar', ...|i_oghUPqLzzJtxC6Z...|Canadian (New), P...|           Montréal|[11:30-22:0, 11:3...|      1|   45.5397329|    -73.633639|            Braseiro|    H2P 2M1|          22|  4.0|   QC|
|2425 Boulevard Go...|[,,,,,,, True,,, ...|735UkyT4px_oeW3iy...|  Parks, Active Life|           Montréal|[9:0-16

We often count things:

In [12]:
result = spark.sql("SELECT count(*) FROM business WHERE state='QC'")
result.show()

+--------+
|count(1)|
+--------+
|    9219|
+--------+



Clean up the column name:

In [13]:
result = spark.sql("SELECT count(*) AS the_count FROM business WHERE state='QC'")
result.show()

+---------+
|the_count|
+---------+
|     9219|
+---------+



Show some grouping (and query formatting):

In [14]:
query = """
SELECT state,count(*) 
    FROM business 
    GROUP BY state
"""
result = spark.sql(query)
result.show()

+-----+--------+
|state|count(1)|
+-----+--------+
|   AZ|   56686|
|   SC|    1162|
|  BAS|       1|
|   NJ|       1|
|   VA|       2|
|   QC|    9219|
|   BC|       1|
|   NV|   36312|
|   WI|    5154|
|   CA|      19|
|   NE|       2|
|   CT|       3|
|   NC|   14720|
|   VT|       2|
|   IL|    1932|
|  XGL|       1|
|   WA|       3|
|   AL|       3|
|  XGM|       4|
|   OH|   14697|
+-----+--------+
only showing top 20 rows



In [15]:
query = """
SELECT state,count(*) as count
    FROM business 
    GROUP BY state
    ORDER BY count(*) DESC
"""
result = spark.sql(query)
result.show()

+-----+-----+
|state|count|
+-----+-----+
|   AZ|56686|
|   NV|36312|
|   ON|33412|
|   NC|14720|
|   OH|14697|
|   PA|11216|
|   QC| 9219|
|   AB| 8012|
|   WI| 5154|
|   IL| 1932|
|   SC| 1162|
|   NY|   22|
|   CA|   19|
|   TX|    6|
|   FL|    4|
|  XGM|    4|
|   WA|    3|
|   AL|    3|
|   CT|    3|
|   NE|    2|
+-----+-----+
only showing top 20 rows



Joining two tables:

In [16]:
query = """
SELECT business.name, tip.text 
    FROM tip 
    LEFT JOIN business 
        ON tip.business_id = business.business_id
"""
result = spark.sql(query)
result.show()

+--------------------+--------------------+
|                name|                text|
+--------------------+--------------------+
|  Buffalo Wild Wings|Great for watchin...|
|      Sonic Drive-In|Happy Hour 2-4 da...|
|Arriba Mexican Grill|Good chips and sa...|
|               Ah-So|The setting and d...|
|            Petagogy|Molly is definate...|
|Yupha's Thai Kitchen|It's true! The dr...|
|      The Black Bull|Only worth a visi...|
|             Fushimi|Hands down best A...|
|           Starbucks|Sat in the drive ...|
|       Brake Masters|Okay so I had a g...|
|Le Montmartre Fre...|More parking in t...|
|       Haus Murphy's|Best pretzels thi...|
|   Krua Thai Cuisine|Having the yellow...|
|  The Captain's Boil|When you receive ...|
|          Burger Bar|2nd stop for the ...|
|            Biên Hòa|Really delicious ...|
| East End Food Co-Op|            Dinner!!|
|   Noodles & Company|Long lines at lun...|
|Findlay Honda Hen...|Go somewhere else...|
|Alberta King of Subs|#poutinecr

## Sampling

In [17]:
query = """
SELECT * FROM checkin TABLESAMPLE (1 PERCENT)
"""
result = spark.sql(query)
result.count()

1623

## User-defined functions

In [18]:
def square(x):
    return x*x

In [19]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
square_udf_int = udf(lambda z: square(z), IntegerType())

In [20]:
spark.udf.register('square_udf_int',square)

<function __main__.square(x)>

In [21]:
query = """
SELECT square_udf_int(compliment_count) AS compliments_squared 
   FROM tip 
   ORDER BY compliments_squared DESC
"""
result = spark.sql(query)
result.show()

+-------------------+
|compliments_squared|
+-------------------+
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
|                  9|
+-------------------+
only showing top 20 rows



## Pandas interoperability

In [22]:
query = """
SELECT state,count(*) as count
    FROM business 
    GROUP BY state
    ORDER BY count(*) DESC
"""
result = spark.sql(query)
result.show()

+-----+-----+
|state|count|
+-----+-----+
|   AZ|56686|
|   NV|36312|
|   ON|33412|
|   NC|14720|
|   OH|14697|
|   PA|11216|
|   QC| 9219|
|   AB| 8012|
|   WI| 5154|
|   IL| 1932|
|   SC| 1162|
|   NY|   22|
|   CA|   19|
|   TX|    6|
|  XGM|    4|
|   FL|    4|
|   WA|    3|
|   AL|    3|
|   CT|    3|
|  XWY|    2|
+-----+-----+
only showing top 20 rows



In [23]:
pandas_result = result.toPandas()

In [24]:
pandas_result

,state,count
0,AZ,56686
1,NV,36312
2,ON,33412
3,NC,14720
4,OH,14697
5,PA,11216
6,QC,9219
7,AB,8012
8,WI,5154
9,IL,1932
